# Building Optimization Models with Gurobipy: A Step-by-Step Guide

## Introduction to Gurobipy

Gurobipy is a Python interface for the Gurobi Optimizer, a powerful commercial solver for various types of optimization problems. This guide walks you through the process of formulating and solving an optimization problem using Gurobipy, based on the furniture production example from our lecture.

## The Furniture Production Problem

Let's recall our problem:

> A furniture company makes tables and chairs.

- Each table requires 4 hours of carpentry and 2 hours of finishing.
- Each chair requires 3 hours of carpentry and 1 hour of finishing.
- The company has 240 hours of carpentry time and 100 hours of finishing time available each week.
- If each table contributes $70 to profit and each chair contributes $50, how many of each should be made to maximize profit?

### Mathematical Formulation

**Decision Variables:**
- x₁ = number of tables to make
- x₂ = number of chairs to make

**Objective:** 
- Maximize profit: 70x₁ + 50x₂

**Constraints:**
- Carpentry time: 4x₁ + 3x₂ ≤ 240
- Finishing time: 2x₁ + x₂ ≤ 100
- Non-negativity: x₁, x₂ ≥ 0

## Step-by-Step Implementation in Gurobipy

### Step 1: Import the Library and Create a Model

In [2]:
import gurobipy as gp
from gurobipy import GRB

# Create a model
model = gp.Model("FurnitureProduction")

Set parameter Username
Set parameter LicenseID to value 2638131
Academic license - for non-commercial use only - expires 2026-03-18


The first step is to import the necessary modules and create a model object. The string parameter gives a name to your model.

### Step 2: Define Decision Variables

In [3]:
# Define decision variables
tables = model.addVar(vtype=GRB.CONTINUOUS, name="Tables")
chairs = model.addVar(vtype=GRB.CONTINUOUS, name="Chairs")

In [5]:
model.update()
tables

<gurobi.Var Tables>

Here we define our decision variables using the `addVar()` method:
- `vtype`: Variable type (continuous, integer, binary)
- `name`: A descriptive name for the variable
- Other parameters (not shown) include `lb` (lower bound), `ub` (upper bound), and `obj` (objective coefficient)

For this problem, we're using continuous variables because we can make fractional numbers of tables and chairs (though in practice, you might want to use integer variables).

### Step 3: Set the Objective Function

In [6]:
# Set objective function
model.setObjective(70 * tables + 50 * chairs, GRB.MAXIMIZE)

We define our objective function using the `setObjective()` method:
- First argument: Linear expression representing the objective function
- Second argument: Direction (GRB.MAXIMIZE or GRB.MINIMIZE)

### Step 4: Add Constraints

In [7]:
# Add constraints
model.addConstr(4 * tables + 3 * chairs <= 240, "CarpentryTime")
model.addConstr(2 * tables + 1 * chairs <= 100, "FinishingTime")
model.addConstr(tables >= 0, "NonNegTables")
model.addConstr(chairs >= 0, "NonNegChairs")

<gurobi.Constr *Awaiting Model Update*>

Constraints are added using the `addConstr()` method:
- First argument: Linear expression representing the constraint
- Second argument: A descriptive name for the constraint

Gurobipy supports the usual comparison operators (<=, >=, ==) in constraints.

### Step 5: Solve the Model

In [8]:
# Solve the model
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Arch Linux")

CPU model: Intel(R) Core(TM) Ultra 9 185H, instruction set [SSE2|AVX|AVX2]
Thread count: 11 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 4 rows, 2 columns and 6 nonzeros
Model fingerprint: 0xd5a2ec53
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [5e+01, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+02]
Presolve removed 2 rows and 0 columns
Presolve time: 0.03s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.2000000e+03   9.974500e+00   0.000000e+00      0s
       2    4.1000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.05 seconds (0.00 work units)
Optimal objective  4.100000000e+03


The `optimize()` method runs the solver on our model.

### Step 6: Access the Results

In [9]:
tables

<gurobi.Var Tables (value 29.999999999999996)>

In [6]:
# Print results
print(f"Optimal value: ${model.objVal:.2f}")
print(f"Tables to produce: {tables.x:.2f}")
print(f"Chairs to produce: {chairs.x:.2f}")

Optimal value: $4100.00
Tables to produce: 30.00
Chairs to produce: 40.00


After optimization:
- `model.objVal` contains the optimal objective value
- `variable.x` contains the optimal value for each variable
- `model.Status` tells you whether the model was solved to optimality

### Step 7: Perform Sensitivity Analysis

In [7]:
# Sensitivity analysis
for constr in model.getConstrs():
    if constr.Sense == '<':  # Only for less-than constraints
        print(f"{constr.ConstrName}: Shadow price = {constr.Pi:.2f}")

CarpentryTime: Shadow price = 15.00
FinishingTime: Shadow price = 5.00


The `Pi` attribute of a constraint gives its shadow price, which represents the marginal value of the associated resource.

## Example Extensions

### Adding New Constraints

You can modify your model by adding new constraints:

In [8]:
# Add a new constraint
model.addConstr(tables <= 35, "MaxTables")

# Re-optimize
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Arch Linux")

CPU model: Intel(R) Core(TM) Ultra 9 185H, instruction set [SSE2|AVX|AVX2]
Thread count: 11 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 5 rows, 2 columns and 7 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [5e+01, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 2e+02]
LP warm-start: use basis

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.1000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.100000000e+03


### Changing to Integer Programming

If we need whole numbers of products:

In [9]:
# Integer variables
tables = model.addVar(vtype=GRB.INTEGER, name="Tables")
chairs = model.addVar(vtype=GRB.INTEGER, name="Chairs")

### Working with Binary Variables

For yes/no decisions:

In [10]:
produce_tables = model.addVar(vtype=GRB.BINARY, name="ProduceTables")

## Common Error Messages and Troubleshooting

1. **Model Infeasible**: No solution satisfies all constraints
   - Check for contradictory constraints
   - Relax some constraints if possible

2. **Model Unbounded**: The objective can be improved indefinitely
   - Verify that all necessary constraints are included
   - Check for incorrect signs in constraints

3. **Numerical Issues**: Solver encounters numerical difficulties
   - Rescale your variables and constraints
   - Avoid very large or very small coefficients

## Practice Exercises

1. Modify the furniture example to include minimum production requirements.
2. Add a constraint that requires at least twice as many chairs as tables.
3. Change the model to use integer variables instead of continuous ones.
4. Add a binary variable to decide whether to produce tables at all.

## Conclusion

Gurobipy provides a powerful and intuitive way to implement optimization models in Python. By following these steps, you can translate a mathematical formulation into working code that solves real-world optimization problems.

The key advantages of using Gurobipy include:
- Clean, intuitive syntax that closely matches mathematical notation
- Access to a high-performance commercial solver
- Extensive modeling capabilities for various problem types
- Robust solution methods for LP, MIP, QP, and other optimization problems
- Advanced features for sensitivity analysis and model debugging